In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from export import update_mpl, figsize, save, DPI
from tools import Figure, Table
update_mpl()

In [ ]:
from siemens import all_versions

In [ ]:
version = next(all_versions())
version.results()

In [ ]:
from tools import make_density, smooth, to_percent

class EffectivenessSummary(Figure):
    def __init__(self, plugins, exam_res=1000, smooth=True):
        self.plugins_ = plugins
        self.exam_res_ = exam_res
        self.smooth_ = smooth
        
    def _render(self):
        data = pd.concat([version.results() for version in all_versions()])
        data = data[data['plugin'].isin(self.plugins_)]
        
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=figsize(1, 0.75), dpi=DPI)
        fig.text(-0.01, 0.5, 'Density of discovered faults', va='center', rotation='vertical')
        
        self._render_ax(ax[0][0], 'exam', 'best', data, show_yticklabels=True)
        # https://stackoverflow.com/questions/4700614/how-to-put-the-legend-out-of-the-plot/43439132#43439132
        fig.legend(
            loc='lower left',
            bbox_to_anchor=(0.1, 0.95, 1, 0.95),
            ncol=len(self.plugins_),
            handlelength=1,
            handletextpad=0.5,
            columnspacing=1)
        
        self._render_ax(ax[0][1], 'exam', 'worst', data)
        self._render_ax(ax[1][0], 'hit', 'best', data, show_yticklabels=True)
        self._render_ax(ax[1][1], 'hit', 'worst', data)
        
        fig.tight_layout()
        
        return fig
        
    def _render_ax(self, ax, metric, kind, data, show_yticklabels=False):
        if metric == 'hit':
            abs_rank_max = data[['best', 'worst']].values.max()
            x = np.linspace(0, abs_rank_max, abs_rank_max)
            ax.set_xlabel(f'Absolute rank ({kind})')
            column = kind
        elif metric == 'exam':
            x = np.linspace(0, 1, self.exam_res_)
            ax.set_xlabel(f'EXAM ({kind})')
            column = f'EXAM {kind}'
            
        grouped = data.groupby('plugin')
        # Use this form of iteration for keeping plugins in specified order.
        for plugin in self.plugins_:
            group = grouped.get_group(plugin)

            y = make_density(group[column].values)(x)
            if self.smooth_:
                y = smooth(y)
            ax.plot(x, y, label=plugin)
        
        if show_yticklabels:
            to_percent(ax, 'y')
        else:
            ax.set_yticklabels([])
        if metric == 'exam':
            to_percent(ax, 'x')

In [ ]:
from tools import fake_handle, to_percent

class Profile(Figure):
    def __init__(self, versions, plugins):
        self.versions_ = versions
        self.plugins_ = plugins
        
    def _render(self):
        data = pd.concat([version.results() for version in self.versions_])
        data = data[data['plugin'].isin(self.plugins_)]
        
        fig = plt.figure(figsize=figsize(0.95, 0.5), dpi=300)
        ax = fig.add_subplot(1, 1, 1)
        
        handles = []
        labels = []
    
        best_style = 'o'
        worst_style = 'D'
        
        x = [version.version_ for version in self.versions_]
        
        grouped = data.groupby('plugin')
        for plugin in self.plugins_:
            group = grouped.get_group(plugin)
            color = next(ax._get_lines.prop_cycler)['color']
            
            y_best = group['EXAM best'].values
            y_worst = group['EXAM worst'].values
            
            ax.scatter(x, y_best, color=color, marker=best_style, s=16)
            ax.scatter(x, y_worst, color=color, marker=worst_style, s=16)
            
            handles.append(fake_handle(color=color))
            labels.append(plugin)
            
        handles.append(fake_handle(marker=best_style))
        labels.append('Best')
    
        handles.append(fake_handle(marker=worst_style))
        labels.append('Worst')
        
        ax.legend(handles, labels)
        ax.grid(False)
    
        ax.set_xlabel('Version')
        ax.set_ylabel('EXAM')
        
        to_percent(ax, 'y')
    
        fig.tight_layout()
        
        return fig

In [ ]:
from tools import make_density

class LiteratureComparison(Table):
    def __init__(self, limits, plugins, literature):
        self.limits_ = np.array(limits)
        self.categories_ = self._get_categories(limits)
        self.plugins_ = plugins
        self.literature_ = literature
        
        self.latex_params = {'float_format': '{:0.2f}'.format}
        
    def _get_categories(self, limits):
        lows = np.roll(limits, 1)
        lows[0] = 0
        return [f'{int(low * 100)}-{int(high * 100)}%' for (low, high) in zip(lows, limits)]
    
    def _categorize(self, results):
        cummulative = make_density(results)(self.limits_)
        shifted = np.roll(cummulative, 1)
        shifted[0] = 0
        return (cummulative - shifted) * 100
    
    def _build(self):
        data = pd.concat([version.results() for version in all_versions()])
        data = data[data['plugin'].isin(self.plugins_)]
        table = self.literature_.copy()
        
        grouped = data.groupby('plugin')
        for plugin in self.plugins_:
            group = grouped.get_group(plugin)
            
            table[f'{plugin} dagger'] = self._categorize(group['EXAM worst'].values)
            
        cols = table.columns.tolist()
        table['EXAM'] = self.categories_
        table = table[['EXAM'] + cols]
            
        return table

In [ ]:
class TechniquesComparison(Table):
    def __init__(self, plugins):
        self.plugins_ = plugins
    
    def _combine_score(self, data, metric):
        combine = lambda best, worst: '{:0.2f} / {:0.2f}'.format(best, worst)
        data[metric] = data.apply(lambda x: combine(x[f'{metric} best'], x[f'{metric} worst']), axis=1)
    
    def _build(self):
        data = pd.concat([version.results() for version in all_versions()])
        data = data[data['plugin'].isin(self.plugins_)]

        data = data.groupby('plugin').mean().mul(100).reset_index()
        data['index'] = data.apply(lambda x: self.plugins_.index(x['plugin']), axis=1)
        extra_cols = [col for col in data.columns if col != 'plugin']
        data = data.set_index('index').sort_index()
        
        self._combine_score(data, 'EXAM')
        self._combine_score(data, 'hit@5')
        self._combine_score(data, 'hit@10')
        
        data = data.reset_index()
        data = data.drop(columns=extra_cols)
        
        return data

In [ ]:
plugins = ['DStar', 'Probabilistic Dependence', 'Likely Invariants']
EffectivenessSummary(plugins).show().save('families-effectiveness.pgf')

In [ ]:
plugins = ['DStar', 'Jaccard', 'Op', 'Ochiai', 'Overlap', 'Tarantula', 'Wong1', 'Zoltar']
EffectivenessSummary(plugins).show().save('sbfl-effectiveness.pgf')

In [ ]:
versions = [version for version in all_versions() if version.program_ == 'schedule2_2.0']
plugins = ['DStar', 'Probabilistic Dependence']
Profile(versions, plugins).show().save('profile.pgf')

In [ ]:
limits = np.array([x / 100 for x in [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]])
plugins = ['Tarantula', 'Probabilistic Dependence']
literature = pd.DataFrame(data={
    'Tarantula': [13.93, 41.80, 5.74, 9.84, 8.20, 7.38, 0.82, 0.82, 4.10, 7.38, 0.00],
    'RankCP best': [41.94, 31.45, 13.71, 2.42, 2.42, 5.65, 1.61, 0.0, 0.8, 0.0, 0.0],
    'RankCP worst': [17.74, 27.42, 25.81, 4.84, 4.84, 8.06, 2.42, 5.65, 2.42, 0.81, 0.0]
})

LiteratureComparison(limits, plugins, literature).show().to_latex()

In [ ]:
plugins = ['DStar', 'Jaccard', 'Op', 'Ochiai', 'Overlap', 'Tarantula', 'Wong1', 'Zoltar', 'Probabilistic Dependence', 'Likely Invariants']
TechniquesComparison(plugins).show().to_latex()